### 상품목록 [] 삭제 후 분석 DB 저장 

In [ ]:
# 먼저 상품목록을 분석 DB에 저장

In [2]:
from DB.db import dbcon

In [ ]:
conn = dbcon()
import re

with conn.cursor() as cur:
    cur.execute("SELECT productID, product_name FROM tb_products")
    result = cur.fetchall()

    for i in result:
        productID = i[0]
        name = i[1]
        clean_name = re.sub("\[.*?\]", "", name)
        print(clean_name)
    #     cur.execute("INSERT INTO tb_분석AI테이블명 (productID, clean_name) VALUES (%s,%s)",(productID, clean_name))
    # conn.commit()

NEW컬러! 카스테라 워싱 옥수수솜 간절기/사계절 차렵이불세트
비침없는 도톰 레이스/쉬폰커튼(나비주름/핀형/봉집)
맞춤 브라우니 린넨 암막커튼 핀형/아일렛형/형상기억 33색상
사각사각 호텔침구 클린코튼 고밀도60수 사계절/간절기 옥수수솜 차렵이불세트 9색상


### 리뷰 내용 중 별점 선택 시 자동으로 들어가는 텍스트 삭제하고 분석 DB 저장

In [ ]:
# 리뷰 내용 중 너무 짧은 내용은 삭제하고 분석 테이블에 리뷰 내용 추가하도록
# 별점-> 최고예요, 마음에 들어요, 보통이에요, 별로예요, 매우 아쉬워요 자동으로 들어가는 텍스트임

In [ ]:
# 제외할 단독 리뷰 문구
exclude_texts = ["최고예요", "마음에 들어요", "보통이에요", "별로예요", "매우 아쉬워요"]

conn = dbcon()

excluded_reviews = []  # 제외된 리뷰 저장
inserted_reviews = []  # 삽입될 리뷰 저장

with conn.cursor() as cur:
    # 원본 리뷰 불러오기
    cur.execute("SELECT reviewID, comment FROM tb_reviews")
    result = cur.fetchall()

    for row in result:
        review_id = row[0]
        review_text = row[1]

        # 제외 문구: 정확히 일치할 때만 제외
        if review_text in exclude_texts:
            excluded_reviews.append((review_id, review_text))
            continue

        inserted_reviews.append((review_id, review_text))

        cur.execute("INSERT INTO tb_analyze (reviewID, comment) VALUES (%s,%s)",(review_id, review_text))
    conn.commit()
# 커밋 전 확인


print("\n삽입될 리뷰:")
for rid, rtext in inserted_reviews:
    print(f"- [{rid}] {rtext}")


삽입될 리뷰:
- [53305893] 제품을 직접 사용해 보니 품질이 기대 이상으로 훌륭합니다. 마감이 깔끔하고 내구성도 좋아 오래 사용할 수 있을 것 같아요. 사용감도 편리하고 성능도 만족스러워서 재구매 의사 100%입니다. 주변에도 자신 있게 추천할 만한 제품이에요.
- [53305745] 색감도 너무 예쁘고 사이즈 적당하고 푹신푹신합니다
- [53291001] 땅에 닿는게 싫어서 쪼금 더 짧게 주문했는데 
길이 너무 맘에 들어요 ㅎㅎㅎ 원하던대로 ㅎㅎ
예쁩니다!!!! 커튼 재질도 좋고 가볍고 너무 좋아여!!!!
- [53290715] 고르고 고르고 또 생각하고 구매했어요.
그런만큼 정말 만족스러워요.
이걸 속지로 쓰고, 겉지를 따로 구매하려했는데
넘 예뻐서 겉지를 안하기로 했네요.
원단도 나비주름도 넘 고급스러워요 ❤️ 
- [53308210] 지금까지 잘쓰고 있네요 포근하고 좋아요~
- [53308125] 선물햇는데 받는 사람이 너무 죠아햇어요
- [53306498] 보들보들하고 세탁기에 돌리기도 편해서 애용 중입니다
- [53306432] 너무예뻐요 ,,, 포근포근해서 잠도 엄청 잘 오고 사계절용이라 에어컨 틀고 여름에 덮어도 괜찮습니다 ~~! 배송도 빨라서 택배 휴무 전에 바로 받을 수 있었어요 ㅎㅎ 
- [53294962] 결제 후에 따로 언제 배송오는지 사전에 연락이 와서 좋았어요. 지그재그로 접혀 배송되었는데 스팀다리미로 슥슥 밀어주니 주름도 많이 사라졌습니다. 혼자서도 레일 달기 쉽게 안내서도 함께 동봉되서 좋았구요~
햇빛은 은은하게 들어오면서 저녁에는 가려줘 프라이버시는 보장되네요
- [53316092] 이불감촉이 부들부들하고 살에 닿는느낌이 좋으네요
- [53313553] 기대한 것보다 더 부드럽고 좋아요 부드러우면서도 냉감이 시원하게 느껴지니 이보다 더 좋을 순 없네요 감사합니다 아이들 하나씩 장만해 줬는데 모두 만족했어요 
- [53313020] 완전 촉감 부드러워요!!
사계절 사용할수 있긴 하지만 한여름에는 좀 더워요.
저희집 사람

In [18]:
len(inserted_reviews)

337

## AI 분석

In [ ]:
# - OpenAI API를 활용해 키워드, 카테고리, 감성 분석 기능 구현.
# - 분석 결과를 DB에 저장하는 함수 작성.

### 리뷰 키워드 추출

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import time

load_dotenv()
prompt = ChatPromptTemplate.from_template(
    """
    당신은 고객 리뷰 데이터를 분석하는 전문가입니다.
    아래 리뷰 텍스트에서 **제품 관련 핵심 명사 키워드 3~5개**를 추출해주세요.

    규칙:
    - 명사만 추출합니다. (예: "색감", "사이즈", "내구성", "마감", "쿠션감")
    - 감정 표현이나 불필요한 단어("좋아요", "최고", "만족")는 제외합니다.
    - 제품의 특징, 장점, 단점, 사용 경험과 관련된 단어를 중심으로 선택합니다.
    - JSON 형식으로 출력합니다.

    리뷰 ID: {review_id}
    리뷰:
    "{review_text}"

    출력 예시:
    {{
      "review_id": {review_id},
      "keywords": ["색감", "사이즈", "내구성"]
    }}
    """
)

llm = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | llm | JsonOutputParser()

all_results = []

# 슬라이싱으로 10개씩 처리
batch_size = 10
for i in range(0, len(inserted_reviews), batch_size):
    batch = inserted_reviews[i : i + batch_size]  # i부터 i+10까지 자르기
    inputs = [{"review_id": rid, "review_text": rtext} for rid, rtext in batch]
    time.sleep(2)
    # LLM batch 호출
    print(f"{i//batch_size + 1}번째 실행 중...")
    results = chain.batch(inputs)
    all_results.extend(results)

# 결과 확인
for r in all_results:
    print(r)

1번째 실행 중...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=4357cc9e-3cf8-498f-9b85-8020937f9ec0,id=4357cc9e-3cf8-498f-9b85-8020937f9ec0; trace=68ca40fc-e62b-43ae-ab9d-d5898c3ff727,id=68ca40fc-e62b-43ae-ab9d-d5898c3ff727; trace=d7447a10-f954-46c6-9473-d330335027bb,id=d7447a10-f954-46c6-9473-d330335027bb; trace=5c96e9dd-fe9b-43fb-bd92-5c164bd4c540,id=5c96e9dd-fe9b-43fb-bd92-5c164bd4c540; trace=4b256a87-8945-4ec2-a943-1a7f1e21549a,id=4b256a87-8945-4ec2-a943-1a7f1e21549a; trace=ac8ec7af-e7ad-45be-81c7-8c04dca634b5,id=ac8ec7af-e7ad-45be-81c7-8c04dca634b5; trace=4ff569b3-fdf7-423f-8d7e-417d59a19feb,id=4ff569b3-fdf7-423f-8d7e-417d59a19feb; trace=b5132ac9-e83c-4b4d-ac33-6ebd09455749

KeyboardInterrupt: 

In [ ]:
all_results

[{'review_id': 53305893, 'keywords': ['품질', '마감', '내구성', '사용감', '성능']},
 {'review_id': 53305745, 'keywords': ['색감', '사이즈', '쿠션감']},
 {'review_id': 53291001, 'keywords': ['길이', '커튼', '재질', '가벼움']},
 {'review_id': 53290715, 'keywords': ['원단', '나비주름', '고급스러움']},
 {'review_id': 53308210, 'keywords': ['포근함']},
 {'review_id': 53308125, 'keywords': []},
 {'review_id': 53306498, 'keywords': ['보들보들함', '세탁기', '편리함']},
 {'review_id': 53306432, 'keywords': ['포근함', '사계절용', '배송']},
 {'review_id': 53294962, 'keywords': ['배송', '주름', '안내서', '햇빛', '프라이버시']},
 {'review_id': 53316092, 'keywords': ['이불', '감촉', '느낌']},
 {'review_id': 53313553, 'keywords': ['부드러움', '냉감', '시원함']},
 {'review_id': 53313020, 'keywords': ['촉감', '여름', '계절', '이불']},
 {'review_id': 53312735, 'keywords': ['온도', '이불', '보온', '간절기', '가을']},
 {'review_id': 53308947, 'keywords': ['원단']},
 {'review_id': 53315265, 'keywords': ['커텐', '사이즈', '천', '나비주름']},
 {'review_id': 53312244, 'keywords': ['채광', '고급스러움', '커텐', '상담']},
 {'review_id': 53310

- 문장별로 카테고리 분류를 해  -> 사용감 
- 그러고 그 카테고리에서 분류된 문장을 보여줘  -> 이불이 폭신해요

- 그다음에 그 카테고리에서 분류된 문장에서 키워드 추출? -> 폭신 

### 리뷰 카테고리 분류

In [ ]:
import re
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")

# 문장별 카테고리 분류
category_prompt = ChatPromptTemplate.from_template(
    """
    You are a classification assistant.
    Classify the given sentence into one of the following categories:

    - 배송: 배송 속도, 포장, 상태와 관련된 내용
    - 사용감: 사용 후 체감, 촉감, 착용감, 사용 경험
    - 사이즈: 크기, 길이, 넓이
    - 디자인: 색상, 모양, 스타일, 미적 요소
    - 품질: 내구성, 마감, 소재, 제품 완성도

    Rules:
    - If no category matches, return "None".
    - Return result in JSON format.

    Input sentence: "{sentence}"
    Output:
    {{
      "sentence": "{sentence}",
      "category": "사용감"
    }}
    """
)
category_chain = category_prompt | llm | JsonOutputParser()

# 키워드 추출
keyword_prompt = ChatPromptTemplate.from_template(
    """
    Extract 1-5 **nouns only** from the following sentence. 
    Exclude general emotion words like "좋아요", "만족", "최고".
    If no obvious product-related noun exists, pick the most meaningful noun in the sentence.
    Return result in JSON format.

    Sentence: "{sentence}"

    Output:
    {{
      "sentence": "{sentence}",
      "keywords": ["배송"]
    }}
    """
)
keyword_chain = keyword_prompt | llm | JsonOutputParser()

# 감성 분석 프롬프트
sentiment_prompt = ChatPromptTemplate.from_template(
    """
    Determine the sentiment of the given sentence.
    Return "긍정", "부정".
    Return result in JSON format.

    Sentence: "{sentence}"

    Output:
    {{
      "sentence": "{sentence}",
      "sentiment": "긍정"
    }}
    """
)
sentiment_chain = sentiment_prompt | llm | JsonOutputParser()

# 실행
all_results = []

for rid, rtext in inserted_reviews:
    # 리뷰를 문장 단위로 분리
    sentences = re.split(r"(?<=[.!?])[\s\n]+", rtext)
    sentences = [s.strip() for s in sentences if s.strip()]  # 빈문장 제거

    for sent in sentences:
        if not sent.strip():
            continue

        # 카테고리 분류
        category_result = category_chain.invoke({"sentence": sent})

        if category_result["category"] != "None":  # 카테고리가 있는 문장만
            # 키워드 추출
            kw_result = keyword_chain.invoke({"sentence": sent})

            # 감성 분석
            sentiment_result = sentiment_chain.invoke({"sentence": sent})

            all_results.append(
                {
                    "review_id": rid,
                    "sentence": sent,
                    "category": category_result["category"],
                    "keywords": kw_result["keywords"],
                    "sentiment": sentiment_result["sentiment"],
                }
            )

# 결과 확인
for r in all_results:
    print(r)

{'review_id': 53305893, 'sentence': '제품을 직접 사용해 보니 품질이 기대 이상으로 훌륭합니다.', 'category': '품질', 'keywords': ['제품', '품질'], 'sentiment': '긍정'}
{'review_id': 53305893, 'sentence': '마감이 깔끔하고 내구성도 좋아 오래 사용할 수 있을 것 같아요.', 'category': '품질', 'keywords': ['마감', '내구성'], 'sentiment': '긍정'}
{'review_id': 53305893, 'sentence': '사용감도 편리하고 성능도 만족스러워서 재구매 의사 100%입니다.', 'category': '사용감', 'keywords': ['사용감', '성능', '재구매', '의사'], 'sentiment': '긍정'}
{'review_id': 53305893, 'sentence': '주변에도 자신 있게 추천할 만한 제품이에요.', 'category': '사용감', 'keywords': ['제품'], 'sentiment': '긍정'}
{'review_id': 53305745, 'sentence': '색감도 너무 예쁘고 사이즈 적당하고 푹신푹신합니다', 'category': '사용감', 'keywords': [], 'sentiment': '긍정'}
{'review_id': 53291001, 'sentence': '땅에 닿는게 싫어서 쪼금 더 짧게 주문했는데 \n길이 너무 맘에 들어요 ㅎㅎㅎ 원하던대로 ㅎㅎ\n예쁩니다!!!!', 'category': '사이즈', 'keywords': ['길이'], 'sentiment': '긍정'}
{'review_id': 53291001, 'sentence': '커튼 재질도 좋고 가볍고 너무 좋아여!!!!', 'category': '품질', 'keywords': ['커튼', '재질'], 'sentiment': '긍정'}
{'review_id': 53290715, 'sentence': '그런만큼 

In [19]:
all_results

[{'review_id': 53305893,
  'sentence': '제품을 직접 사용해 보니 품질이 기대 이상으로 훌륭합니다.',
  'category': '품질',
  'keywords': ['제품', '품질'],
  'sentiment': '긍정'},
 {'review_id': 53305893,
  'sentence': '마감이 깔끔하고 내구성도 좋아 오래 사용할 수 있을 것 같아요.',
  'category': '품질',
  'keywords': ['마감', '내구성'],
  'sentiment': '긍정'},
 {'review_id': 53305893,
  'sentence': '사용감도 편리하고 성능도 만족스러워서 재구매 의사 100%입니다.',
  'category': '사용감',
  'keywords': ['사용감', '성능', '재구매', '의사'],
  'sentiment': '긍정'},
 {'review_id': 53305893,
  'sentence': '주변에도 자신 있게 추천할 만한 제품이에요.',
  'category': '사용감',
  'keywords': ['제품'],
  'sentiment': '긍정'},
 {'review_id': 53305745,
  'sentence': '색감도 너무 예쁘고 사이즈 적당하고 푹신푹신합니다',
  'category': '디자인',
  'keywords': ['색감', '사이즈'],
  'sentiment': '긍정'},
 {'review_id': 53291001,
  'sentence': '땅에 닿는게 싫어서 쪼금 더 짧게 주문했는데 \n길이 너무 맘에 들어요 ㅎㅎㅎ 원하던대로 ㅎㅎ\n예쁩니다!!!!',
  'category': '사이즈',
  'keywords': ['길이'],
  'sentiment': '긍정'},
 {'review_id': 53291001,
  'sentence': '커튼 재질도 좋고 가볍고 너무 좋아여!!!!',
  'category': '품질',
  'keywords